In [1]:
import simpy
import pandas as pd
import matplotlib.pyplot as plt
import PyQt5
%matplotlib qt5

In [71]:
wood_capacity = 1000
initial_wood = 500
dispatch_capacity = 500
guitars_made=0
#env生成一个仿真虚拟环境
env=simpy.Environment()
#定义wood材料仓库，喷涂前仓库(琴身)，喷涂后仓库(琴身)，喷涂前仓库(琴颈)，喷涂后仓库(琴颈)，电子元器件仓库的,dispatch成本仓库初始值与最大值
class Guitar_Factory:
    def __init__(self, env):
        self.wood = simpy.Container(env, capacity = wood_capacity, init = initial_wood)
        self.body_pre_paint=simpy.Container(env,capacity=100,init=0)
        self.body_post_paint=simpy.Container(env,capacity=200,init=0)
        self.neck_pre_paint=simpy.Container(env,capacity=100,init=0)
        self.neck_post_paint=simpy.Container(env,capacity=200,init=0)
        self.electronic=simpy.Container(env,capacity=100,init=100)
        self.dispatch = simpy.Container(env ,capacity = dispatch_capacity, init = 0)
        self.wood_control=env.process(self.wood_stock_control(env))
        self.electronic_control=env.process(self.electronic_stock_control(env))
        self.dispatch_control=env.process(self.dispatch_guitars_control(env))
        self.env_status_monitor = env.process(self.env_status(env))
    #wood材料仓库存量监控    
    def wood_stock_control(self, env):
        while True:
            #安全库存标准
            wood_critial_stock=100
            if self.wood.level <= wood_critial_stock:
                print('在第{0}日 第{1}小时，木材库存 ({2})低于预警库存水平下 '.format(
                    int(env.now/8), env.now % 8,self.wood.level))
                print('联系供应商')
                print('----------------------------------')
                yield env.timeout(16)
                print('在第{0}天 第{1}小时，木材送达300'.format(int(env.now/8), env.now % 8))
                yield self.wood.put(300)
                print('当前库存是：{0}'.format(
                    self.wood.level))
                print('----------------------------------')
                yield env.timeout(8)
            else:
                yield env.timeout(1)
            #安全库存标准
    def electronic_stock_control(self, env):
        while True:
            electronic_critial_stock=50
            if self.electronic.level <= electronic_critial_stock:
                print('在第{0}日 第{1}小时，电子器件库存 ({2})低于预警库存水平下 '.format(
                    int(env.now/8), env.now % 8,self.electronic.level))
#                 print('联系供应商')
#                 print('----------------------------------')
#                 yield env.timeout(16)
#                 print('在第{0}天 第{1}小时，木材送达300'.format(int(env.now/8), env.now % 8))
#                 yield self.wood.put(300)
                print('当前电子元器件库存是：{0}'.format(
                    self.electronic.level))
                print('----------------------------------')
                yield env.timeout(8)
            else:
                yield env.timeout(1)    
    def dispatch_guitars_control(self, env):
        global guitars_made
        yield env.timeout(0)
        while True:
            if self.dispatch.level >= 50:
                print('成品库存为：{0}, 在第{1}日 第{2}小时 联系了商场取货'.format(self.dispatch.level, int(env.now/8), env.now % 8))
                print('----------------------------------')
                yield env.timeout(4)
                print('在第{0}日 第{1}小时，商场取走{2}吉他'.format(int(env.now/8), env.now % 8,self.dispatch.level))
                guitars_made += self.dispatch.level
                yield self.dispatch.get(self.dispatch.level)
                print('----------------------------------')
                yield env.timeout(8)
            else:
                yield env.timeout(1)
    def env_status(self, env):
        global status
        status = pd.DataFrame(columns = ["datetime", "dispatch_level",'wood','electronic','body_pre_paint','neck_pre_paint','body_post_paint','neck_post_paint'])
        status[["datetime", "dispatch_level",'wood','electronic','body_pre_paint','neck_pre_paint','body_post_paint','neck_post_paint']] = status[["datetime", "dispatch_level",'wood','electronic','body_pre_paint','neck_pre_paint','body_post_paint','neck_post_paint']].astype(int)
        while True:
            plt.ion()
            plt.cla()
            plt.plot(status['datetime'], status['neck_pre_paint'],color='#4D9221')
            plt.title('neck_pre_paint')
            plt.pause(0.05)
            print('{0}在第{1}日 第{2}小时，成品库存量：{3}'.format(env.now,int(env.now/8), env.now % 8,self.dispatch.level))
            tmp = {'datetime':env.now,
                   'dispatch_level':self.dispatch.level,
                   'wood':self.wood.level,
                   'electronic':self.electronic.level,
                   'body_pre_paint':self.body_pre_paint.level,
                   'neck_pre_paint':self.neck_pre_paint.level,
                   'body_post_paint':self.body_post_paint.level,
                   'neck_post_paint':self.neck_post_paint.level
                   }
            status = status.append([tmp])
            plt.ioff()
            yield env.timeout(1)

In [72]:
#实例化工厂
guitar_factory=Guitar_Factory(env)

In [93]:
#container的level属性-》当前存货水平
guitar_factory.wood.level

500

In [141]:
guitar_factory.dispatch.level

0

In [77]:
#container的capacity属性最大值
guitar_factory.wood.capacity

1000

In [78]:
guitar_factory.dispatch.capacity

500

In [73]:
#yield作为资源调用的赋值作用，给事件赋值
#get为container的方法表示拿出
#put为container的方法表示拿进
#body maker为制造琴身，neck maker为制作琴颈，painter为喷涂程序,assembler为组装程序
def body_maker(env, guitar_factory):
    while True:
        yield guitar_factory.wood.get(1)
        #制作时间1小时
        body_time = 1
        yield env.timeout(body_time)
        yield guitar_factory.body_pre_paint.put(1)

def neck_maker(env, guitar_factory):
    while True:
        yield guitar_factory.wood.get(1)
        neck_time = 1
        yield env.timeout(neck_time)
        yield guitar_factory.neck_pre_paint.put(2)
def painter(env,guitar_factory):
    while True:
        #一次喷涂10个
        yield guitar_factory.body_pre_paint.get(5)
        yield guitar_factory.neck_pre_paint.get(5)
        yield env.timeout(10)
        yield guitar_factory.body_post_paint.put(5)
        yield guitar_factory.neck_post_paint.put(5)
def assembler(env,guitar_factory):
    while True:
        yield guitar_factory.body_post_paint.get(1)
        yield guitar_factory.neck_post_paint.get(1)
        yield guitar_factory.electronic.get(1)
        yield env.timeout(1)
        yield guitar_factory.dispatch.put(1)

In [74]:
#two events create for body and neck 
body_maker_process=env.process(body_maker(env,guitar_factory))
neck_maker_process=env.process(neck_maker(env,guitar_factory))
painter_process=env.process(painter(env,guitar_factory))
assembler_process=env.process(assembler(env,guitar_factory))
# control_process=env.process(guitar_factory.wood_stock_control(env))
# dispatch_process=env.process(guitar_factory.dispatch_guitars_control(env))

In [75]:
#仿真运行40个小时
env.run(until=8*50)

0在第0日 第0小时，成品库存量：0
1在第0日 第1小时，成品库存量：0
2在第0日 第2小时，成品库存量：0
3在第0日 第3小时，成品库存量：0
4在第0日 第4小时，成品库存量：0
5在第0日 第5小时，成品库存量：0
6在第0日 第6小时，成品库存量：0
7在第0日 第7小时，成品库存量：0
8在第1日 第0小时，成品库存量：0
9在第1日 第1小时，成品库存量：0
10在第1日 第2小时，成品库存量：0
11在第1日 第3小时，成品库存量：0
12在第1日 第4小时，成品库存量：0
13在第1日 第5小时，成品库存量：0
14在第1日 第6小时，成品库存量：0
15在第1日 第7小时，成品库存量：0
16在第2日 第0小时，成品库存量：0
17在第2日 第1小时，成品库存量：1
18在第2日 第2小时，成品库存量：2
19在第2日 第3小时，成品库存量：3
20在第2日 第4小时，成品库存量：4
21在第2日 第5小时，成品库存量：5
22在第2日 第6小时，成品库存量：5
23在第2日 第7小时，成品库存量：5
24在第3日 第0小时，成品库存量：5
25在第3日 第1小时，成品库存量：5
26在第3日 第2小时，成品库存量：5
27在第3日 第3小时，成品库存量：6
28在第3日 第4小时，成品库存量：7
29在第3日 第5小时，成品库存量：8
30在第3日 第6小时，成品库存量：9
31在第3日 第7小时，成品库存量：10
32在第4日 第0小时，成品库存量：10
33在第4日 第1小时，成品库存量：10
34在第4日 第2小时，成品库存量：10
35在第4日 第3小时，成品库存量：10
36在第4日 第4小时，成品库存量：10
37在第4日 第5小时，成品库存量：11
38在第4日 第6小时，成品库存量：12
39在第4日 第7小时，成品库存量：13
40在第5日 第0小时，成品库存量：14
41在第5日 第1小时，成品库存量：15
42在第5日 第2小时，成品库存量：15
43在第5日 第3小时，成品库存量：15
44在第5日 第4小时，成品库存量：15
45在第5日 第5小时，成品库存量：15
46在第5日 第6小时，成品库存量：15
47在第5日 第7小时，成品库存量：16
48在第6日 第0小时，成品库存量：17
49在第6日 第1小时，

282在第35日 第2小时，成品库存量：0
283在第35日 第3小时，成品库存量：0
284在第35日 第4小时，成品库存量：0
285在第35日 第5小时，成品库存量：0
在第35日 第6小时，电子器件库存 (0)低于预警库存水平下 
当前电子元器件库存是：0
----------------------------------
286在第35日 第6小时，成品库存量：0
287在第35日 第7小时，成品库存量：0
288在第36日 第0小时，成品库存量：0
289在第36日 第1小时，成品库存量：0
290在第36日 第2小时，成品库存量：0
291在第36日 第3小时，成品库存量：0
292在第36日 第4小时，成品库存量：0
293在第36日 第5小时，成品库存量：0
在第36日 第6小时，电子器件库存 (0)低于预警库存水平下 
当前电子元器件库存是：0
----------------------------------
294在第36日 第6小时，成品库存量：0
295在第36日 第7小时，成品库存量：0
296在第37日 第0小时，成品库存量：0
297在第37日 第1小时，成品库存量：0
298在第37日 第2小时，成品库存量：0
299在第37日 第3小时，成品库存量：0
300在第37日 第4小时，成品库存量：0
301在第37日 第5小时，成品库存量：0
在第37日 第6小时，电子器件库存 (0)低于预警库存水平下 
当前电子元器件库存是：0
----------------------------------
302在第37日 第6小时，成品库存量：0
303在第37日 第7小时，成品库存量：0
304在第38日 第0小时，成品库存量：0
305在第38日 第1小时，成品库存量：0
306在第38日 第2小时，成品库存量：0
307在第38日 第3小时，成品库存量：0
308在第38日 第4小时，成品库存量：0
309在第38日 第5小时，成品库存量：0
在第38日 第6小时，电子器件库存 (0)低于预警库存水平下 
当前电子元器件库存是：0
----------------------------------
310在第38日 第6小时，成品库存量：0
311在第38日 第7小时，成品库存量：0
312在第39日 第0小时，成品库存量：

In [151]:
guitar_factory.dispatch.level

100

In [152]:
guitar_factory.wood.level

348

In [154]:
env.now

400

### ZL 物流

In [2]:
import numpy as np
import simpy
env=simpy.Environment()
date_list=[]

In [24]:
from TM1py.Services import TM1Service
tm1 = TM1Service(address="localhost", port=30059, ssl=False, user="admin", password="Apple")
mdx='''
  SELECT 
  NON EMPTY 
   {Tm1filterbylevel(Descendants([Month Day].[All Months]),0)} 
  ON COLUMNS , 
  NON EMPTY 
   {Tm1filterbylevel([Product].members,0)} 
   PROPERTIES [Product].[Product].[中文]  ON ROWS 
FROM [ZL Amazon Payments Data STG] 
WHERE 
  (
   [Scenario].[Scenario].[ACT],
   [Version].[Version].[WIP],
   [Order Year].[Order Year].[2021],
   [COA Company].[COA Company].[002],
   [Currency].[Currency].[USD],
   [Country].[Country].[US],
   [Platform].[Platform].[03],
   [Data Source Payments Data STG].[Data Source Payments Data STG].[All Datasources],
   [Behavior].[Behavior].[商品销售],
   [SA Year].[SA Year].[2021],
   [M ZL Amazon Payments Data STG].[M ZL Amazon Payments Data STG].[Quantity],
   [Region State].[Region State].[All Region State List],
   [Region City].[Region City].[All Region City List],
   [SA Month].[SA Month].[All MTDs]
  )
'''
df=tm1.cubes.cells.execute_mdx_dataframe(mdx=mdx)
df_sales=df[df['Product']=='LSKU1571']
df_sales.reset_index(inplace=True,drop=True)
df_sales['Value']=df_sales['Value'].astype('int')

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [25]:
df_sales

,Product,Month Day,Value
0,LSKU1571,01-03,1
1,LSKU1571,01-04,25
2,LSKU1571,01-05,47
3,LSKU1571,01-06,46
4,LSKU1571,01-07,51
...,...,...,...
587,LSKU1571,12-27,6
588,LSKU1571,12-28,22
589,LSKU1571,12-29,46
590,LSKU1571,12-30,51


In [57]:
class Port_Stock:
    def __init__(self, env,df:'DataFrame'):
        #国内口岸仓库
        self.domestic_port = simpy.Container(env, capacity = 999999999, init = 0)
        #美国口岸仓库
        self.us_port=simpy.Container(env, capacity = 999999999, init = 0)
        #亚马逊03站仓库
        self.us_amazon_03=simpy.Container(env, capacity = 999999999, init = 5000)
        #日期进行时
        self.date_process=env.process(self.date(env,df))
        #安全库存监控
        self.stock_control_process=env.process(self.lsku_stock_control(env,df))
    def date(self,env,df:'DataFrame'):
        global date_list
        while True:
            print('现在日期是{0}'.format(df.iloc[env.now,1]))
            date_list.append(df.iloc[env.now,1])
            yield env.timeout(1)
    def lsku_stock_control(self,env,df):
        while True:
            #安全库存标准
            LSKU_critial_stock=2000
            if self.us_amazon_03.level <= LSKU_critial_stock:
                print('在第{0}日 LSKU1571库存 ({1})低于预警库存水平下 '.format(df.iloc[env.now,1],self.us_amazon_03.level))
                print('需联系供应商')
                print('----------------------------------')
                #domestic_transport_process=env.process(self.domestic_transport(env,df))
                #LSKU1571 需要7天到达国内港口
                yield env.timeout(7)
                #np.random.normal(300,50,1)
                #进货5000支
                candle_arrive=5000
                yield self.domestic_port.put(candle_arrive)
                print('在第{0}日，国内港口LSKU1571蜡烛存货到达{1}支，货物开始打包运输'.format(df.iloc[env.now,1],candle_arrive))
                self.domestic_port.get(candle_arrive)
                #运输7天到达美国港口
                yield env.timeout(7)
                yield self.us_port.put(candle_arrive)
                
                
                
                #us_transport_process=env.process(self.us_transport(env,df))
                candle_arrive=5000
                yield self.us_port.get(candle_arrive)
                #LSKU1571 需要7天到达亚马逊仓库 
                yield env.timeout(7)
                yield self.us_amazon_03.put(candle_arrive)
                print('env.now是',env.now)
                print('在第{0}日，LSKU1571蜡烛{1}支到达亚马逊03站仓库'.format(df.iloc[env.now,1],candle_arrive))  
                yield env.timeout(22)
            else:
                yield env.timeout(1)
#     def stock_fee_calculate(self,env,stockrate):
#         while True:
#             total_fee=0
#             total_fee=total_fee+self.us_amazon_03.level*stockrate
#             print('在第{0}日，LSKU1571蜡烛在亚马逊03站仓库的总费用{1}'.format(df.iloc[env.now,1],total_fee))  
#             yield env.timeout(1)
#     def domestic_transport(self,env,df:'DataFrame'):
#         while True:
#             #LSKU1571 需要7天到达国内港口
#             yield env.timeout(7)
#             #np.random.normal(300,50,1)
#             #进货5000支
#             candle_arrive=5000
#             yield self.domestic_port.put(candle_arrive)
#             print('在第{0}日，国内港口LSKU1571蜡烛存货到达{1}支，货物开始打包运输'.format(df.iloc[env.now,1],candle_arrive))
#             self.domestic_port.get(candle_arrive)
#             #运输7天到达美国港口
#             yield env.timeout(7)
#             yield self.us_port.put(candle_arrive)
#     def us_transport(self,env,df:'DataFrame'):
#         while True:
#             candle_arrive=5000
#             yield self.us_port.get(candle_arrive)
#             #LSKU1571 需要7天到达亚马逊仓库 
#             yield env.timeout(7)
#             yield self.us_amazon_03.put(candle_arrive)
#             print('在地{1}日，LSKU1571蜡烛{2}支到达亚马逊03站仓库'.format(df.iloc[env.now,1],candle_arrive))         
        

In [19]:
def selling(env,port_stock,df:'DataFrame'):
    while True:    
        sales=df.iloc[env.now,2]
        yield port_stock.us_amazon_03.get(sales)
        print('在第{0}日，亚马逊03站仓库LSKU1571蜡烛余量{1}'.format(df.iloc[env.now,1],port_stock.us_amazon_03.level))
        yield env.timeout(1)
        
    
    
    

In [58]:
#实例化
date_list=[]
env=simpy.Environment()
port_stock=Port_Stock(env,df_sales)

In [59]:
selling_process=env.process(selling(env,port_stock,df_sales))

In [60]:
env.run(until=150)

现在日期是01-03
在第01-03日，亚马逊03站仓库LSKU1571蜡烛余量4999
现在日期是01-04
在第01-04日，亚马逊03站仓库LSKU1571蜡烛余量4974
现在日期是01-05
在第01-05日，亚马逊03站仓库LSKU1571蜡烛余量4927
现在日期是01-06
在第01-06日，亚马逊03站仓库LSKU1571蜡烛余量4881
现在日期是01-07
在第01-07日，亚马逊03站仓库LSKU1571蜡烛余量4830
现在日期是01-08
在第01-08日，亚马逊03站仓库LSKU1571蜡烛余量4794
现在日期是01-09
在第01-09日，亚马逊03站仓库LSKU1571蜡烛余量4764
现在日期是01-10
在第01-10日，亚马逊03站仓库LSKU1571蜡烛余量4746
现在日期是01-11
在第01-11日，亚马逊03站仓库LSKU1571蜡烛余量4741
现在日期是01-12
在第01-12日，亚马逊03站仓库LSKU1571蜡烛余量4739
现在日期是01-13
在第01-13日，亚马逊03站仓库LSKU1571蜡烛余量4737
现在日期是01-14
在第01-14日，亚马逊03站仓库LSKU1571蜡烛余量4735
现在日期是01-15
在第01-15日，亚马逊03站仓库LSKU1571蜡烛余量4734
现在日期是01-16
在第01-16日，亚马逊03站仓库LSKU1571蜡烛余量4733
现在日期是01-17
在第01-17日，亚马逊03站仓库LSKU1571蜡烛余量4730
现在日期是01-18
在第01-18日，亚马逊03站仓库LSKU1571蜡烛余量4729
现在日期是01-19
在第01-19日，亚马逊03站仓库LSKU1571蜡烛余量4728
现在日期是01-20
在第01-20日，亚马逊03站仓库LSKU1571蜡烛余量4727
现在日期是01-23
在第01-23日，亚马逊03站仓库LSKU1571蜡烛余量4726
现在日期是01-26
在第01-26日，亚马逊03站仓库LSKU1571蜡烛余量4725
现在日期是01-29
在第01-29日，亚马逊03站仓库LSKU1571蜡烛余量4722
现在日期是02-02
在第02-02日，亚马逊03站仓库LSKU1571蜡烛余量4721
现在日期是02-04

In [27]:
port_stock.us_amazon_03.level

4999

In [30]:
for i in df_sales.index:
    df.iloc[i,1]
    print(df.iloc[i,1])

01-03
01-04
01-05
01-06
01-07
01-08
01-09
01-10
01-11
01-12
01-13
01-14
01-15
01-16
01-17
01-18
01-19
01-20
01-23
01-26
01-29
02-02
02-04
02-08
02-09
02-11
02-12
02-13
02-14
02-15
02-16
02-17
02-18
02-19
02-20
02-21
02-22
02-23
02-24
02-25
02-26
02-27
02-28
03-01
03-02
03-03
03-04
03-05
03-06
03-07
03-08
03-09
03-10
03-11
03-12
03-13
03-14
03-15
03-16
03-17
03-20
03-21
03-22
03-25
03-26
03-27
03-30
03-31
04-02
04-04
04-07
04-08
04-09
04-10
04-11
04-12
04-13
04-14
04-15
04-16
04-17
04-18
04-19
04-20
04-21
04-22
04-23
04-24
04-25
04-26
04-27
04-28
04-29
04-30
05-01
05-02
05-03
05-04
05-05
05-06
05-07
05-08
05-09
05-10
05-11
05-12
05-13
05-14
05-15
05-16
05-17
05-18
05-19
05-20
05-21
05-22
05-23
05-24
05-25
05-26
05-27
05-28
05-29
05-31
06-01
06-02
06-03
06-08
06-09
06-10
06-12
06-18
06-19
06-22
06-28
06-30
07-01
07-02
07-04
07-07
07-11
07-14
07-16
07-17
07-18
07-22
07-23
07-24
07-25
07-26
07-27
07-28
07-29
07-30
07-31
08-01
08-02
08-03
08-04
08-05
08-06
08-07
08-08
08-09
08-10
08-11
08-1

In [52]:
df_sales

,Product,Month Day,Value
0,LSKU1571,01-03,1
1,LSKU1571,01-04,25
2,LSKU1571,01-05,47
3,LSKU1571,01-06,46
4,LSKU1571,01-07,51
...,...,...,...
587,LSKU1571,12-27,6
588,LSKU1571,12-28,22
589,LSKU1571,12-29,46
590,LSKU1571,12-30,51


In [26]:
df[df['Product']=='LSKU1571'].iloc[0,1]592 rows × 3 columns

'01-03'

In [27]:
a=[]
a.append('1-2')
a

['1-2']

In [98]:
1<=2
1<=2
1<=2
1<=2

True